In [1]:
from bs4 import BeautifulSoup
from time import sleep
import requests
import pandas as pds

In [2]:
#edit this for your own search
baseurl = "www.indeed.com/jobs?q=machine%20learning%20intern"
soupurls = [baseurl]

#change range to numjobs / 15
#there are 15 listings per page but for some reason the url uses &start in increments of 10
for i in range(1,1264//15):
    soupurls.append(baseurl + "&start=%i" % (i*10))

In [3]:
def scrapeJobIds(soup):
    jk = [str(soup).find("jk:")]
    for i in range(0,15):
        jk.append(str(soup).find("jk:", jk[i]+1))
        
    jobs = []
    for i in jk:
        jobs.append(str(soup)[i+4:i+20])
        
    return jobs

In [4]:
ids = []
for url in soupurls:
    #if scraping a lot of pages you may want to have breaks between requests
    #sleep(2) 
    soup = BeautifulSoup(requests.get("https://" + url).text, 'html.parser')
    ids.append(scrapeJobIds(soup))

In [9]:
ids = sum(ids, [])

TypeError: can only concatenate list (not "str") to list

In [6]:
soups = []
for i in ids:
    url = "https://www.indeed.com/viewjob?jk=%s" % i
    
    #if scraping a lot of pages you may want to have breaks between requests
    #sleep(2) 
    page = requests.get(url)
    soups.append(BeautifulSoup(page.text, 'html.parser'))

In [7]:
titles = [] 
description = []

for soup in soups:
    titles.append(soup.find("title").get_text())
    try:
        description.append(soup.find("div", "jobsearch-jobDescriptionText").get_text().strip())
    except:
        description.append("No description")
        
data = pds.DataFrame({"title:": titles, "description:": description})

In [8]:
#change string to whatever name you want
data.to_csv("data.csv")

In [10]:
pds.DataFrame(ids).to_csv("jobids.csv")

In [ ]:
for i in list(ids):
    if 'OCTYPE' in i:
        ids.remove(i)

In [14]:
'OCTYPE' in ids[975]

True

In [13]:
ids[975]

'OCTYPE html>\n\n<h'